In [1]:
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Rectangle
import numpy as np

from functions import win_loss

In [2]:

week_by_week2024 = nfl.import_weekly_data([2024], downcast=True)
play_by_play2024 = nfl.import_pbp_data([2024], downcast=True, cache=False, alt_path=None)
schedule2024 = nfl.import_schedules([2024])
seasonal2024 = nfl.import_seasonal_data([2024], "REG")
winstotal2024 = nfl.import_win_totals([2024])

Downcasting floats.
2024 done.
Downcasting floats.


In [ ]:
# printing the columns of play_byPlay so I don't mess up names lol
for column in play_by_play2024.columns:
    print(column)

In [ ]:
for column in schedule2024.columns:
    print(column)

In [28]:
# printing the columns of week_by_week so I don't mess up names lol
for columns in week_by_week2024.columns: 
    print (columns)

# testing    
print(week_by_week2024['position'].unique())
print(week_by_week2024['position_group'].unique())

week_by_week2024_fp = week_by_week2024[week_by_week2024['fantasy_points_ppr'] > 0]
week_by_week2024_fp #2715 

player_id
player_name
player_display_name
position
position_group
headshot_url
recent_team
season
week
season_type
opponent_team
completions
attempts
passing_yards
passing_tds
interceptions
sacks
sack_yards
sack_fumbles
sack_fumbles_lost
passing_air_yards
passing_yards_after_catch
passing_first_downs
passing_epa
passing_2pt_conversions
pacr
dakota
carries
rushing_yards
rushing_tds
rushing_fumbles
rushing_fumbles_lost
rushing_first_downs
rushing_epa
rushing_2pt_conversions
receptions
targets
receiving_yards
receiving_tds
receiving_fumbles
receiving_fumbles_lost
receiving_air_yards
receiving_yards_after_catch
receiving_first_downs
receiving_epa
receiving_2pt_conversions
racr
target_share
air_yards_share
wopr
special_teams_tds
fantasy_points
fantasy_points_ppr
['QB' 'TE' 'P' 'FS' 'FB' 'WR' 'RB' 'CB' 'T' 'OLB' 'SS' 'G' 'ILB']
['QB' 'TE' 'SPEC' 'DB' 'RB' 'WR' 'OL' 'LB']


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,8.580000,8.580000
1,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,2,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,15.140000,15.140000
2,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,3,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,21.040001,21.040001
3,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,4,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,11.600000,11.600000
4,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,5,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,11.760000,11.760000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,3,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.800000,0.800000
2711,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,4,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,5.000000,5.000000
2712,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,6,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,2.600000,2.600000
2713,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,8,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,1.000000,1.000000


In [ ]:
smallWeekByWeek_df =week_by_week2024[['position', 'position_group']]

# getting fantasy defense ranking through week by week data
def fantasy_defense_rankings(week_by_week_df): 
    smallFantasy_df = week_by_week_df[['fantasy_points_ppr', 'position', 'opponent_team', 'player_name', 'week']]
    releventPositions_df = smallFantasy_df[smallFantasy_df['position'].isin(['QB', 'RB', 'WR', 'TE'])]
    
    # using top performers b/c thats usually more indicative of sos in fantasy football 
    topPerformers_df = releventPositions_df.sort_values(by='fantasy_points_ppr', ascending=False).groupby(['opponent_team', 'position']).head(3)
    qb_df = topPerformers_df[topPerformers_df["position"] == "QB"].sort_values(by='fantasy_points_ppr', ascending=False).groupby(['opponent_team', 'position']).head(1)
    
    
    # groupy by opponent team and position in order to find out how many total fantasy points each position has scored against them 
    defensePointsAgainst = topPerformers_df.groupby(['opponent_team', 'position'])['fantasy_points_ppr'].mean().reset_index()
    qbPointsAgainst = qb_df.groupby(['opponent_team', 'position'])['fantasy_points_ppr'].mean().reset_index()
    
    # assign a rank to each team based on the total fantasy points scored against them
    defensePointsAgainst['rank'] = defensePointsAgainst.groupby('position')['fantasy_points_ppr'].rank(ascending=True)
    
    defensePointsAgainst = defensePointsAgainst.sort_values(by=['position', 'rank'])
    
    return qbPointsAgainst

test = fantasy_defense_rankings(week_by_week2024)
test = test.sort_values(by='fantasy_points_ppr', ascending=True)
test.head(15)

,opponent_team,position,fantasy_points_ppr
26,PIT,QB,16.379999
17,LAC,QB,20.740000
21,NE,QB,21.040001
10,DET,QB,21.600000
19,MIA,QB,22.180000
5,CHI,QB,22.240000
23,NYG,QB,22.760000
11,GB,QB,23.320000
9,DEN,QB,23.600000
3,BUF,QB,23.639999


In [ ]:
# creating a function to calculate the win/loss ratio for each team given the seasonal data
def win_loss (schedule_df): 
    # making the dataframe smaller so its easier to handle the data
    smallSchedule2024 = schedule_df[['result', 'away_team', 'away_score', 'home_team', 'home_score', 'week']]   
    previousGames = smallSchedule2024[schedule_df['result'].notnull()]
    
    
    previousGames['home_win'] = np.where(previousGames['result'] > 0, 1, 0)
    previousGames['away_win'] = np.where(previousGames['result'] < 0, 1, 0)
    
    ## not sure if all this is necessary. I think the best way to do it would be to just count the 1s and 0s to figure out the record
    # obtaining the week list
    weekList = previousGames['week'].unique()
    
    #going week by week to calculate the records
    for week in weekList: 
        # getting the data for each week at a time (to tally up the win/loss at the specific time of playing)
        previousWeekData = previousGames[previousGames['week'] <= week]
        
        # calculating the wins and losses for each team
        homeWins = previousWeekData.groupby('home_team')['home_win'].sum()
        awayWins = previousWeekData.groupby('away_team')['away_win'].sum()
        homeLosses = previousWeekData.groupby('home_team')['home_win'].count() - previousWeekData.groupby('home_team')['home_win'].sum()
        awayLosses = previousWeekData.groupby('away_team')['away_win'].count() - previousWeekData.groupby('away_team')['away_win'].sum()
        
        teamWins = homeWins.add(awayWins, fill_value=0)
        teamLosses = homeLosses.add(awayLosses, fill_value=0)
        
        # creates a mask so we get the right week in the origianl df
        mask = schedule_df['week'] == week

        # adding the win/loss record to the schedule_df
        schedule_df.loc[mask, 'home_team_win_record'] = schedule_df['home_team'].map(teamWins)
        schedule_df.loc[mask, 'home_team_loss_record'] = schedule_df['home_team'].map(teamLosses)
        schedule_df.loc[mask, 'away_team_win_record'] = schedule_df['away_team'].map(teamWins)
        schedule_df.loc[mask, 'away_team_loss_record'] = schedule_df['away_team'].map(teamLosses)

    return schedule_df


scheduleWithRecords = win_loss(schedule2024) 
previousGames = scheduleWithRecords[scheduleWithRecords['result'].notnull()]
previousGames
